# Bilan Hydrique Climatologie

In [1]:
import numpy as np
import pandas as pd

# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]
# Altitude (m)
REF_STATION_ALTITUDE = 50.

# Identification de l'API Météo-France
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

# Météo-France API
METEOFRANCE_API = 'DPClim'

# Départements où chercher des stations
ID_DEPARTEMENTS = [35, 50]

# Fréquence des données climatiques
METEOFRANCE_FREQUENCE = 'quotidienne'

# NN_NOMBRE = 20
# NN_RAYON_KM = None
NN_NOMBRE = None
NN_RAYON_KM = 35.

In [2]:
import meteofrance

# Initialisation d'un client pour accéder à l'API Météo-France
client = meteofrance.Client(APPLICATION_ID, METEOFRANCE_API)

# Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': '*/*'})

In [4]:
LECTURE_LISTE_STATIONS = False

l_listes = []
for id_dep in ID_DEPARTEMENTS:
    filepath_liste_stations = meteofrance.get_filepath_liste_stations(
        client, id_dep)
    
    if LECTURE_LISTE_STATIONS:
        # Lecture par département
        filepath_liste_stations = meteofrance.get_filepath_liste_stations(
            client, id_dep)
        df_liste_stations_dep = pd.read_csv(filepath_liste_stations, index_col=0)
    else:
        # Demande de la liste des stations pour le département
        section = meteofrance.SECTION_LISTE_STATIONS
        params = {'id-departement': id_dep}
        response = meteofrance.demande(
            client, section, params=params, frequence=METEOFRANCE_FREQUENCE)
        df_liste_stations_dep = meteofrance.response_text_to_frame(
            client, response, index_col=client.id_station_label)

        # Sauvegarde par département
        df_liste_stations_dep.to_csv(filepath_liste_stations)

    # Liste pour compilation
    l_listes.append(df_liste_stations_dep)

# Compilation
df_liste_stations = pd.concat(l_listes, axis='index')
df_liste_stations

,nom,posteOuvert,typePoste,lon,lat,alt,postePublic
id,,,,,,,
35002001,AMANLIS-SPC,True,5,-1.473500,48.009500,42,True
35004001,ANTRAIN,False,4,-1.481667,48.463333,25,True
35005001,ARBRISSEL,True,1,-1.297333,47.927333,75,True
35012001,BAIN-DE-B.,False,4,-1.686667,47.841667,103,True
35012002,BAIN-DE-BRETAGNE BORG,False,4,-1.681667,47.841667,108,True
...,...,...,...,...,...,...,...
50615001,VALOGNES,False,2,-1.506667,49.517667,61,True
50615002,VALOGNES BOURG,False,4,-1.466667,49.508333,40,True
50616001,LE VAL-SAINT-PERE BOUILLE,False,4,-1.375000,48.663333,3,True


In [5]:
import geo

df_liste_stations_nn = geo.selection_plus_proches_voisins(
    df_liste_stations, REF_STATION_LATLON, client.latlon_labels,
    nombre=NN_NOMBRE, rayon_km=NN_RAYON_KM)

df_liste_stations_nn

/home/atantet/Documents/Travail/1_agri/agri-notebooks/geo.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liste_stations_nn.loc[:, 'distance'] = dist_km


,nom,posteOuvert,typePoste,lon,lat,alt,postePublic,distance
id,,,,,,,,
35339001,TRANS BOURG,False,4,-1.586667,48.495000,100,True,6
35044001,BROUALAN,True,2,-1.640833,48.485667,99,True,6
35270002,ST-GEORGES,False,3,-1.539167,48.572667,6,True,7
50410001,PONTORSON AGGLOMERATION,False,4,-1.508333,48.556667,19,True,8
35095002,DOL-DE-BRETAGNE,False,2,-1.731667,48.548333,19,True,9
...,...,...,...,...,...,...,...,...
50218005,GRANVILLE VILLE,False,4,-1.585000,48.840000,66,True,33
50218007,GRANVILLE-STEP,False,4,-1.568333,48.845333,24,True,34
35228001,DINARD,True,0,-2.076333,48.584833,65,True,34


In [5]:
print(f'{REF_STATION_NAME} ({REF_STATION_LATLON[0]:.6f}, {REF_STATION_LATLON[1]:.6f}) est à :')
for k, (nn_id, nn_series) in enumerate(df_liste_stations_nn.transpose().items()):
    station_name = df_liste_stations_nn.loc[nn_id, client.station_name_label]
    print(f'- {df_liste_stations_nn.loc[nn_id, 'distance']:02d} km du {k}NN {station_name} '
          f'({nn_series[client.latlon_labels[0]]:.6f}, '
          f'{nn_series[client.latlon_labels[1]]:.6f})')

La Petite Claye (48.541356, -1.615400) est à :
- 06 km du 0NN TRANS BOURG (48.495000, -1.586667)
- 06 km du 1NN BROUALAN (48.485667, -1.640833)
- 07 km du 2NN ST-GEORGES (48.572667, -1.539167)
- 08 km du 3NN PONTORSON AGGLOMERATION (48.556667, -1.508333)
- 09 km du 4NN DOL-DE-BRETAGNE (48.548333, -1.731667)
- 09 km du 5NN CHERRUEIX (48.611667, -1.676667)
- 09 km du 6NN PONTORSON (48.576667, -1.503333)
- 09 km du 7NN CHERRUEIX (48.607833, -1.692833)
- 09 km du 8NN PONTORSON (48.585667, -1.505167)
- 10 km du 9NN DOL-DE-BRETAGNE (48.550000, -1.750000)
- 10 km du 10NN BEAUVOIR BOURG (48.596667, -1.501667)
- 12 km du 11NN BONNEMAIN (48.473333, -1.736667)
- 12 km du 12NN BONNEMAIN (48.473333, -1.736667)
- 13 km du 13NN BAZOUGES-LA-PEROUSE (48.426667, -1.573333)
- 13 km du 14NN ANTRAIN (48.463333, -1.481667)
- 14 km du 15NN NOYAL-SOUS-B. (48.414000, -1.617000)
- 15 km du 16NN LOURMAIS (48.428333, -1.738333)
- 16 km du 17NN SAINT-OUEN-LA-ROUERIE BOURG (48.461667, -1.440000)
- 17 km du 18NN SAI

In [6]:
DATE_DEB_PERIODE = '2024-12-01T00:00:00Z'
DATE_FIN_PERIODE = '2024-12-31T23:00:00Z'

id_commandes = meteofrance.compiler_commandes_des_stations_periode(
    client, df_liste_stations_nn.iloc[:2], DATE_DEB_PERIODE, DATE_FIN_PERIODE,
    frequence=METEOFRANCE_FREQUENCE)
id_commandes

{35339001: '789988018251', 35044001: '789988018380'}

In [8]:
section = 'commande'
response = meteofrance.demande(client, section, {'id-cmde': '789988018251'}, frequence='fichier')
response

HTTPError: 500 Server Error: Internal Server Error for url: https://public-api.meteofrance.fr/public/DPClim/v1/commande/fichier?id-cmde=789988018251